In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
from google.colab import files
!kaggle config set -n path -v{/content}
%env KAGGLE_USERNAME = nadaessam1
%env KAGGLE_KEY = 4c585acd6813fde431ebeb93c04385d1
!kaggle competitions download -c fcis-sc-deeplearning-competition -p/content
!unzip \*.zip

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl
- path is now set to: {/content}
env: KAGGLE_USERNAME=nadaessam1
env: KAGGLE_KEY=4c585acd6813fde431ebeb93c04385d1
fcis-sc-deeplearning-competition.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  fcis-sc-deeplearning-competition.zip
replace submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: test/test/1412_mb_file_0a8c5_gif.jpg  


In [ ]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
import tflearn
from sklearn.model_selection import train_test_split
from __future__ import division, print_function, absolute_import

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, pooling, AveragePooling2D, InputLayer, GlobalAveragePooling2D
#from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from google.colab.patches import cv2_imshow
from keras import optimizers
import csv
from keras.optimizers import Adam, rmsprop, sgd
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def write_in_file(prediction):
  num = 0

  with open('submit.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['id', 'label'])

  for i in os.listdir(test_dir):
    if i[-1] != 'g':
      continue
  
    p = np.argmax(prediction[num])
    print(p)
    name_of_class = class_names[p]
    print(name_of_class)
    print(class_ids[name_of_class], '\n')
    row = [i, class_ids[name_of_class]]
    with open('submit.csv', 'a') as fd:
      writer = csv.writer(fd)
      writer.writerow(row)
    num += 1
  

  row = ['1412_mb_file_0a8c5_gif.jpg', '4']
  with open('submit.csv', 'a') as fd:
    writer = csv.writer(fd)
    writer.writerow(row)
  print('finished writing')

In [ ]:

class_ids = {'airport_inside':1, 'bakery':2, 'bedroom':3, 'greenhouse':4, 'gym':5, 'kitchen':6, 'operating_room':7, 'poolinside':8, 'restaurant':9, 'toystore':10}
img_size = 224

reading data

In [ ]:
train_dir = 'train/train'
test_dir = 'test/test'
class_names = {}
print('starting')
def read_train_data():
    
    train_data = []
    class_number = 0
    for i in tqdm(os.listdir(train_dir)):
        folder_path = os.path.join(train_dir, i)
        print(i)
        class_names[class_number] = i
        for j in os.listdir(folder_path):
            img_path = os.path.join(folder_path, j)
            #print(img_path)
            if img_path[-1] != 'g' or j == 'indooPool_Inside_gif.jpg':
              continue
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = img[:,:, [2,1,0]]
            img = cv2.resize(img, (img_size, img_size))
            train_data.append([img, class_number])

        class_number += 1
    shuffle(train_data)
    np.save('train_data.npy', train_data)
    return train_data


#if os.path.exists('train_data.npy'):
    #data = np.load('train_data.npy', allow_pickle=True)
    

#else:
data = read_train_data()

X = np.array([i[0] for i in data]).reshape(-1, img_size, img_size, 3)

Y = [i[1] for i in data]
    




  0%|          | 0/10 [00:00<?, ?it/s]

starting
kitchen


 10%|█         | 1/10 [00:04<00:42,  4.75s/it]

bedroom


 20%|██        | 2/10 [00:12<00:45,  5.68s/it]

...

 80%|████████  | 8/10 [00:21<00:03,  1.75s/it]

airport_inside


 90%|█████████ | 9/10 [00:25<00:02,  2.64s/it]

bakery


100%|██████████| 10/10 [00:28<00:00,  2.52s/it]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [ ]:
def read_test_data():
  test_data = []
  if os.path.exists('test/test/1412_mb_file_0a8c5_gif.jpg'):
    os.unlink('test/test/1412_mb_file_0a8c5_gif.jpg')
  for i in tqdm(os.listdir(test_dir)):
    img_path = os.path.join(test_dir, i)
    
    if img_path[-1] != 'g':
      continue
    
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = img[:,:, [2,1,0]]
    img = cv2.resize(img, (img_size, img_size))
    test_data.append(img)
  np.save('test_data.npy', test_data)
  return test_data

test_data = read_test_data()


100%|██████████| 771/771 [00:07<00:00, 102.85it/s]


data augmentation

defining Model

In [ ]:
'''from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
'''
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
#from tensorflow.python.keras.applications.mobilenet import MobileNet, preprocess_input
#from tensorflow.python.keras.applications.densenet import DenseNet201, preprocess_input
#from tensorflow.python.keras.applications.densenet import DenseNet169, preprocess_input
#from tensorflow.python.keras.applications.vgg19 import VGG19, preprocess_input
#from tensorflow.python.keras.applications.xception import Xception, preprocess_input

#from tensorflow.python.keras.applications.nasnet import NASNetMobile, preprocess_input
#from tensorflow.python.keras.applications.resnet import ResNet152, preprocess_input
#from tensorflow.python.keras.applications.resnet import ResNet50, preprocess_input


preprocessing

In [ ]:

X_train_proc = preprocess_input(X_train)

X_test_proc = preprocess_input(X_test)
#whole data
X_proc = preprocess_input(X)
Y_hot = keras.utils.to_categorical(Y, num_classes=10)

train_hot = keras.utils.to_categorical(Y_train, num_classes=10)
test_hot = keras.utils.to_categorical(Y_test, num_classes=10)

build model

In [ ]:
#imported_model = InceptionV3(include_top=False, weights='imagenet')
imported_model = InceptionResNetV2(include_top=False, weights='imagenet')
#imported_model = VGG19(include_top=False, weights='imagenet')
#imported_model = Xception(include_top=False, weights='imagenet')
#imported_model = NASNetMobile(include_top=False, weights='imagenet')
#imported_model = ResNet50(include_top=False, weights='imagenet')
#imported_model = InceptionV3(include_top=False, weights='imagenet')
#imported_model = MobileNet(include_top=False, weights='imagenet')
#imported_model = DenseNet201(include_top=False, weights='imagenet')


219062272/219055592 [==============================] - 17s 0us/step


In [ ]:
#imported_model.summary()

In [ ]:
pred_train = imported_model.predict(X_train_proc)
pred_test = imported_model.predict(X_test_proc)

pred_Whole_data = imported_model.predict(X_proc)

In [ ]:
built_model = Sequential()
built_model.add(GlobalAveragePooling2D())

built_model.add(Dense(1024, activation='relu'))
built_model.add(Dense(10, activation='softmax'))

In [ ]:
built_model.compile(loss='categorical_crossentropy', optimizer=rmsprop(0.00001), metrics=['categorical_accuracy'])
#history = built_model.fit(pred_train, np.array(train_hot), epochs=30, validation_data=(pred_test, np.array(test_hot)))
history = built_model.fit(pred_Whole_data, np.array(Y_hot), epochs=30)

Epoch 1/30
3135/3135 [==============================] - 10s 3ms/step - loss: 1.3704 - categorical_accuracy: 0.6520
Epoch 2/30
3135/3135 [==============================] - 1s 212us/step - loss: 0.6877 - categorical_accuracy: 0.8335
Epoch 3/30
3135/3135 [==============================] - 1s 219us/step - loss: 0.4858 - categorical_accuracy: 0.8791
Epoch 4/30
3135/3135 [==============================] - 1s 212us/step - loss: 0.3987 - categorical_accuracy: 0.8951
Epoch 5/30
3135/3135 [==============================] - 1s 207us/step - loss: 0.3525 - categorical_accuracy: 0.9043
Epoch 6/30
3135/3135 [==============================] - 1s 216us/step - loss: 0.3213 - categorical_accuracy: 0.9116
Epoch 7/30
3135/3135 [==============================] - 1s 208us/step - loss: 0.2995 - categorical_accuracy: 0.9152
Epoch 8/30
3135/3135 [==============================] - 1s 205us/step - loss: 0.2810 - categorical_accuracy: 0.9203
Epoch 9/30
3135/3135 [==============================] - 1s 216us/step - l

In [ ]:
test_data = np.array([i for i in test_data]).reshape(-1, img_size, img_size, 3)
test_data_proc = preprocess_input(test_data)

pred_test2 = imported_model.predict(test_data_proc)
pred_test_onModel = built_model.predict(pred_test2)

write_in_file(pred_test_onModel)

8
airport_inside
1 

6
toystore
10 

4
poolinside
8 

2
gym
5 

2
gym
5 

8
airport_inside
1 

3
restaurant
9 

1
bedroom
3 

1
bedroom
3 

2
gym
5 

8
airport_inside
1 

9
bakery
2 

9
bakery
2 

6
toystore
10 

3
restaurant
9 

0
kitchen
6 

8
airport_inside
1 

2
gym
5 

6
toystore
10 

1
bedroom
3 

0
kitchen
6 

0
kitchen
6 

0
kitchen
6 

1
bedroom
3 

0
kitchen
6 

3
restaurant
9 

4
poolinside
8 

8
airport_inside
1 

9
bakery
2 

1
bedroom
3 

9
bakery
2 

9
bakery
2 

8
airport_inside
1 

3
restaurant
9 

5
operating_room
7 

0
kitchen
6 

5
operating_room
7 

8
airport_inside
1 

9
bakery
2 

0
kitchen
6 

1
bedroom
3 

8
airport_inside
1 

8
airport_inside
1 

8
airport_inside
1 

3
restaurant
9 

6
toystore
10 

7
greenhouse
4 

9
bakery
2 

8
airport_inside
1 

0
kitchen
6 

8
airport_inside
1 

1
bedroom
3 

2
gym
5 

1
bedroom
3 

0
kitchen
6 

1
bedroom
3 

1
bedroom
3 

3
restaurant
9 

9
bakery
2 

9
bakery
2 

8
airport_inside
1 

4
poolinside
8 

7
greenhouse
4 

9

fine tuning

In [ ]:
imported_model.summary()


Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 

In [ ]:

F_final_model = Sequential()
F_final_model.add(imported_model)
#for l in imported_model.layers:
  #F_final_model.layers.append(l)

F_final_model.add(GlobalAveragePooling2D())
F_final_model.add(Dense(1024, activation='relu'))
F_final_model.add(Dense(10, activation='softmax'))

F_final_model.layers[0].trainable = True
print(len(imported_model.layers))
for l in F_final_model.layers[:18]:
  l.trainable = False

#for l in F_final_model.layers[10:]:
  #l.trainable = True
'''
x = imported_model.output
x = (GlobalAveragePooling2D())
x = Dense(1024, activation='relu')
x = (Dense(10, activation='softmax'))

F_final_model = Model(input=imported_model.input, output=x)

F_final_model.layers[0].trainable = True

for l in F_final_model.layers[0].layers[:-300]:
  l.trainable = False
for l in F_final_model.layers[0].layers[-11:]:
  l.trainable = True
'''
for l in F_final_model.layers:
  print(l.trainable)

TypeError: ignored

In [ ]:
F_final_model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
F_final_model.fit(X_train_proc, np.array(train_hot), epochs=1000, validation_data=(X_test_proc, np.array(test_hot)))
#F_final_model.fit(X_proc, np.array(Y_hot), epochs=30) #validation_data=(X_test_proc, np.array(test_hot))


In [ ]:
F_test_data_pred = F_final_model.predict(test_data_proc)
write_in_file(F_test_data_pred)